# 6D scan setup/calibration

In [ ]:
import sys
import os
from os.path import join
from pprint import pprint
import importlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import pandas as pd
import h5py
from scipy import ndimage
import proplot as pplt

sys.path.append('..')
from tools import plotting as mplt
from tools import utils

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['cmap.discrete'] = False

## Load data 

In [ ]:
datadir = '../Diagnostics/Data/Measurements/scan6d/2022-06-23/'
filenames = os.listdir(datadir)
filenames

In [ ]:
_filenames = dict()
_filenames['background'] = '220623100854-scan6d-background.h5'
_filenames['screen_response'] = '220623100854-scan6d-screenResponse.h5'
_filenames['phase_calibration'] = '220623100854-scan6d-phaseCalibration.h5'

In [ ]:
def load_data(filename):
    file = h5py.File(join(datadir, filename), 'r')
    data = file['scandata']
    pprint(data.dtype.fields)

    # Errors and warnings from log
    for i in range(file['log'].size):
        if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
            timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
            print(f"{timestr} {file['log'][i, 'message']}")

    # Configuration data
    for key in file['/config'].keys():
        print(f"{key}")
        print("--------------")
        for name in file['/config'][key].dtype.names:
            print(f"{name}: {file['config'][key][name]}")
        print()
    return data

## Background 

In [ ]:
data = load_data(_filenames['background'])

In [ ]:
for i in range(min(len(data), 50)):
    fig, ax = pplt.subplots(figsize=(6.0, 1.0))
    ax.plot(data[i, 'cam08_ProfileY'], color='black')
    plt.show()

In [ ]:
# [...]

## Screen response 

In [ ]:
data = load_data(_filenames['screen_response'])

In [ ]:
for name in ['iteration', 'cam08_Saturation', 'steerV_Readback', 'steerV_Set']:
    fig, ax = pplt.subplots(figsize=(7.0, 1.25))
    ax.plot(data[name], color='black')
    ax.format(ylabel=name, xlabel='Step')
    plt.show()

In [ ]:
iteration_start = 2
profiles = []
steerVs = []
for iteration in range(iteration_start, max(data['iteration']) + 1):
    idx = data['iteration'] == iteration
    profile = np.mean(data[idx, 'cam08_ProfileY'], axis=0)
    profiles.append(profile)
    steerV = np.mean(data[idx, 'steerV_Readback'])
    steerVs.append(steerV)

In [ ]:
scale = 0.5 * np.max(profiles)
fig, ax = pplt.subplots()
for i in range(len(profiles)):
    offset = i * scale
    ax.plot(profiles[i] + offset, color='black')

In [ ]:
fig, ax = pplt.subplots()
profiles = np.array(profiles)
ax.pcolormesh(np.arange(profiles.shape[1]), steerVs, profiles)
ax.format(ylabel='screen x pixel', xlabel='steerV')

In [ ]:
# [...]

## Phase calibration 

In [ ]:
data = load_data(_filenames['phase_calibration'])

In [ ]:
for name in ['iteration', 'phase_Readback', 'phase_Set']:
    fig, ax = pplt.subplots(figsize=(7.0, 1.5))
    ax.plot(data[name], color='black')
    ax.format(ylabel=name, xlabel='Step')
    plt.show()

In [ ]:
iteration_start = 1
profiles = []
phase_rbs = []
for iteration in range(iteration_start, max(data['iteration']) + 1):
    idx = data['iteration'] == iteration
    profile = np.mean(data[idx, 'cam08_ProfileY'], axis=0)
    profiles.append(profile)
    phase_rb = np.mean(data[idx, 'phase_Readback'])
    phase_rbs.append(phase_rb)

In [ ]:
scale = 0.5 * np.max(profiles)
fig, ax = pplt.subplots()
for i in range(len(profiles)):
    offset = i * scale
    ax.plot(profiles[i] + offset, color='black')

In [ ]:
fig, ax = pplt.subplots()
profiles = np.array(profiles)
ax.pcolormesh(np.arange(profiles.shape[1]), phase_rbs, profiles, colorbar=True, cmap='mono')
ax.format(ylabel='screen x pixel', xlabel='phase_rb')